In [ ]:
!pip install nsepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36085 sha256=c09453ad0438b4f68fc93ed61dd295efbb0125b2ad1d05ae0991f9c7304e0145
  Stored in directory: /root/.cache/pip/wheels/b9/2c/cb/d74199e422a1dbc20297d10cfff9c54d03e0b5c6694d49ccdd
Successfully built nsepy


In [ ]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from nsepy import get_history
from datetime import date
import plotly.graph_objs as go
from datetime import datetime

In [ ]:
nse_data = pd.read_csv('/content/drive/MyDrive/MDG SOC 23/NSE Stock Data(upto 21feb).csv')
nse_data.head()

,Unnamed: 0,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,0,20MICRONS,2008-10-06,40.00,40.00,15.800,16.775,14.731860,23501600.0
1,1,20MICRONS,2008-10-07,16.00,19.00,13.925,15.025,13.195003,9113400.0
2,2,20MICRONS,2008-10-08,14.00,14.60,12.550,13.250,11.636194,2464384.0
3,3,20MICRONS,2008-10-10,12.45,12.45,10.825,11.600,10.187158,1207928.0
4,4,20MICRONS,2008-10-13,12.15,13.30,11.650,12.325,10.823855,898692.0


In [ ]:
nse_data.tail()

,Unnamed: 0,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
5958299,5958299,ZYDUSWELL,2023-02-16,1467.000000,1474.000000,1430.150024,1437.599976,1437.599976,8726.0
5958300,5958300,ZYDUSWELL,2023-02-17,1444.800049,1463.949951,1436.250000,1451.349976,1451.349976,7297.0
5958301,5958301,ZYDUSWELL,2023-02-20,1459.000000,1459.750000,1440.949951,1445.449951,1445.449951,15578.0
5958302,5958302,ZYDUSWELL,2023-02-21,1453.050049,1479.000000,1441.150024,1462.300049,1462.300049,16177.0
5958303,5958303,ZYDUSWELL,2023-02-22,1462.300049,1462.300049,1437.800049,1443.000000,1443.000000,31103.0


In [ ]:
nse_data.columns

Index(['Unnamed: 0', 'Ticker', 'Date', 'Open', 'High', 'Low', 'Close',
       'Adj Close', 'Volume'],
      dtype='object')

In [ ]:
nse_data.drop('Unnamed: 0',axis='columns',inplace=True)
nse_data


,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,20MICRONS,2008-10-06,40.000000,40.000000,15.800000,16.775000,14.731860,23501600.0
1,20MICRONS,2008-10-07,16.000000,19.000000,13.925000,15.025000,13.195003,9113400.0
2,20MICRONS,2008-10-08,14.000000,14.600000,12.550000,13.250000,11.636194,2464384.0
3,20MICRONS,2008-10-10,12.450000,12.450000,10.825000,11.600000,10.187158,1207928.0
4,20MICRONS,2008-10-13,12.150000,13.300000,11.650000,12.325000,10.823855,898692.0
...,...,...,...,...,...,...,...,...
5958299,ZYDUSWELL,2023-02-16,1467.000000,1474.000000,1430.150024,1437.599976,1437.599976,8726.0
5958300,ZYDUSWELL,2023-02-17,1444.800049,1463.949951,1436.250000,1451.349976,1451.349976,7297.0
5958301,ZYDUSWELL,2023-02-20,1459.000000,1459.750000,1440.949951,1445.449951,1445.449951,15578.0
5958302,ZYDUSWELL,2023-02-21,1453.050049,1479.000000,1441.150024,1462.300049,1462.300049,16177.0


In [ ]:
nse_tickdate=pd.DataFrame(columns=['Ticker','Date'])
nse_tickdate['Ticker']=nse_data['Ticker']
nse_tickdate['Date']=nse_data['Date']
nse_tickdate

,Ticker,Date
0,20MICRONS,2008-10-06
1,20MICRONS,2008-10-07
2,20MICRONS,2008-10-08
3,20MICRONS,2008-10-10
4,20MICRONS,2008-10-13
...,...,...
5958299,ZYDUSWELL,2023-02-16
5958300,ZYDUSWELL,2023-02-17
5958301,ZYDUSWELL,2023-02-20
5958302,ZYDUSWELL,2023-02-21


In [ ]:
nse_tickdate.drop_duplicates(subset='Ticker',keep='first',inplace=True)
nse_tickdate

,Ticker,Date
0,20MICRONS,2008-10-06
3546,21STCENMGM,2018-01-11
4811,360ONE,2019-09-19
5664,3IINFOLTD,2005-04-22
10069,3MINDIA,2002-07-01
...,...,...
5940796,ZOTA,2017-05-10
5942230,ZUARI,2012-11-27
5944758,ZUARIIND,2002-07-01
5949890,ZYDUSLIFE,2002-07-01


In [ ]:
nse_tickdate.reset_index(inplace=True)
nse_tickdate

,index,Ticker,Date
0,0,20MICRONS,2008-10-06
1,3546,21STCENMGM,2018-01-11
2,4811,360ONE,2019-09-19
3,5664,3IINFOLTD,2005-04-22
4,10069,3MINDIA,2002-07-01
...,...,...,...
1886,5940796,ZOTA,2017-05-10
1887,5942230,ZUARI,2012-11-27
1888,5944758,ZUARIIND,2002-07-01
1889,5949890,ZYDUSLIFE,2002-07-01


In [ ]:
nse_tickdate['Ticker'][0]

'20MICRONS'

In [ ]:
ticker=input("Enter symbol of stock\n")
spos=0
# nse_input_data=pd.DataFrame(column=['Ticker','Date','Open','High','Low','Close','Adj Close','Volume'])
for i in range(0,len(nse_tickdate)):
  if(nse_tickdate['Ticker'][i]==ticker):
    spos=nse_tickdate['Date'][i]
spos

Enter symbol of stock
ADANIENT


'2002-07-01'

In [ ]:
print(type(spos))


spos = datetime.strptime(spos, '%Y-%m-%d').date()
print(type(spos))

<class 'str'>
<class 'datetime.date'>


In [ ]:
spos

datetime.date(2002, 7, 1)

In [ ]:
nse_input_data = get_history(symbol=ticker, start=spos, end=date.today())


In [ ]:
print(type(nse_input_data))
nse_input_data.columns

<class 'pandas.core.frame.DataFrame'>


Index(['Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last',
       'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume',
       '%Deliverble'],
      dtype='object')

In [ ]:
nse_input_data

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2002-07-01,ADANIEXPO,EQ,183.20,178.00,182.90,174.10,179.00,177.80,177.06,82799,1.466065e+12,NaN,16342.0,0.1974
2002-07-02,ADANIEXPO,EQ,177.80,180.90,181.50,177.00,179.50,179.15,179.68,77875,1.399264e+12,NaN,2895.0,0.0372
2002-07-03,ADANIEXPO,EQ,179.15,179.00,181.00,178.10,179.00,179.15,180.14,75135,1.353477e+12,NaN,1695.0,0.0226
2002-07-04,ADANIEXPO,EQ,179.15,180.00,182.85,180.00,180.00,180.20,182.04,74549,1.357055e+12,NaN,3210.0,0.0431
2002-07-05,ADANIEXPO,EQ,180.20,181.15,181.15,179.10,180.00,179.75,179.57,74683,1.341061e+12,NaN,6912.0,0.0926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-27,ADANIENT,EQ,1315.65,1300.00,1313.80,1131.05,1187.55,1193.50,1222.37,10271008,1.255494e+15,487401.0,2037372.0,0.1984
2023-02-28,ADANIENT,EQ,1193.50,1145.00,1398.00,1103.75,1371.35,1363.85,1307.03,33948366,4.437163e+15,1047492.0,7489663.0,0.2206
2023-03-01,ADANIENT,EQ,1363.85,1424.00,1580.00,1411.00,1579.00,1564.30,1511.37,20114340,3.040028e+15,758963.0,2222197.0,0.1105


In [ ]:
nse_input_data['Date']=nse_input_data.index
nse_input_data

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Date
Date,,,,,,,,,,,,,,,
2002-07-01,ADANIEXPO,EQ,183.20,178.00,182.90,174.10,179.00,177.80,177.06,82799,1.466065e+12,NaN,16342.0,0.1974,2002-07-01
2002-07-02,ADANIEXPO,EQ,177.80,180.90,181.50,177.00,179.50,179.15,179.68,77875,1.399264e+12,NaN,2895.0,0.0372,2002-07-02
2002-07-03,ADANIEXPO,EQ,179.15,179.00,181.00,178.10,179.00,179.15,180.14,75135,1.353477e+12,NaN,1695.0,0.0226,2002-07-03
2002-07-04,ADANIEXPO,EQ,179.15,180.00,182.85,180.00,180.00,180.20,182.04,74549,1.357055e+12,NaN,3210.0,0.0431,2002-07-04
2002-07-05,ADANIEXPO,EQ,180.20,181.15,181.15,179.10,180.00,179.75,179.57,74683,1.341061e+12,NaN,6912.0,0.0926,2002-07-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-27,ADANIENT,EQ,1315.65,1300.00,1313.80,1131.05,1187.55,1193.50,1222.37,10271008,1.255494e+15,487401.0,2037372.0,0.1984,2023-02-27
2023-02-28,ADANIENT,EQ,1193.50,1145.00,1398.00,1103.75,1371.35,1363.85,1307.03,33948366,4.437163e+15,1047492.0,7489663.0,0.2206,2023-02-28
2023-03-01,ADANIENT,EQ,1363.85,1424.00,1580.00,1411.00,1579.00,1564.30,1511.37,20114340,3.040028e+15,758963.0,2222197.0,0.1105,2023-03-01


In [ ]:

# Create a line chart using Plotly
fig = go.Figure(data=go.Scatter(x=nse_input_data['Date'], y=nse_input_data['Close'], line=dict(color='blue', width=2)))
fig.update_layout(title=ticker, xaxis_title='Date', yaxis_title='Price')
fig.update_xaxes(dtick='M12')
# Show the chart
fig.show()


In [ ]:
fig = go.Figure(data=[go.Candlestick(x=nse_input_data['Date'],
                open=nse_input_data['Open'],
                high=nse_input_data['High'],
                low=nse_input_data['Low'],
                close=nse_input_data['Close'])])


fig.update_layout(
    title=ticker,
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    height=600,
    xaxis=dict(type='category',categoryorder='category ascending'),

)




fig.show()
